In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime
import geopandas as gpd
import seaborn as sns
import plotly.express as px

In [2]:
pd.set_option('display.max_columns', None)
faults = pd.read_csv('../data/J1939Faults.csv', low_memory = False )
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv', low_memory = False)
service_fault_code = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

/Applications/anaconda/anaconda3/envs/geospatial_ds6/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [4]:
faults.dtypes 

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
dtype: object

In [5]:
diagnostics

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [6]:
diagnostics['Name'].unique()

array(['IgnStatus', 'EngineOilPressure', 'EngineOilTemperature',
       'TurboBoostPressure', 'EngineLoad', 'AcceleratorPedal',
       'IntakeManifoldTemperature', 'FuelRate', 'FuelLtd', 'EngineRpm',
       'LampStatus', 'BarometricPressure', 'FuelLevel', 'Speed',
       'EngineTimeLtd', 'CruiseControlSetSpeed', 'CruiseControlActive',
       'EngineCoolantTemperature', 'ParkingBrake',
       'SwitchedBatteryVoltage', 'DistanceLtd', 'Throttle',
       'FuelTemperature', 'ServiceDistance'], dtype=object)

In [7]:
diagnostics['Name'].nunique()

24

In [8]:
service_fault_code 

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


#### 36.0666667, -86.4347222 = 193 Aldi Blvd, Mount Juliet, TN  37122, United States
#### 35.5883333, -86.4438888 = 119–121 Eady Rd, Shelbyville, TN  37160, United States
#### 36.1950, -83.174722 = 5754 Fish Hatchery Rd, Russellville, TN  37860, United States

In [9]:
# Define the service locations
service_locations = [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]

# Filter out faults occurring within 1 mile of service locations
for loc in service_locations:
    lat_mile = 2/69 # Convert 1 mile to degrees of latitude
    lon_mile =  1/(69 * np.cos(np.radians(loc[0]))) * 2 # Convert 2 mile to degrees of longitude at the given latitude
    faults = faults[(np.abs(faults['Latitude'] - loc[0]) > lat_mile) | (np.abs(faults['Longitude'] - loc[1]) > lon_mile)]
    
# Remove rows with Equipment ID > 5
faults = faults[faults['EquipmentID'].str.len() <= 5]
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [10]:
# pivot the diagnostics dataframe
wide_diag = diagnostics.pivot(index='FaultId', columns='Name', values='Value')
wide_diag

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96


In [11]:
# Merge fault and diagnostics dataframe
fault_diag = pd.merge(left = faults,
         right = wide_diag,
         left_on = 'RecordID',
         right_on = 'FaultId'
        )
fault_diag

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051207,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1051208,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1051209,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1051210,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,863.25,586.75,23.6,4042.49282573,0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [12]:
fault_diag['EquipmentID'].nunique()

1043

In [13]:
fault_diag['EquipmentID'].unique()

array(['1439', '1369', '1674', ..., '2356', '2380', '2381'], dtype=object)

In [14]:
fault_diag['spn'].unique()

array([   111,    629,   1807,   4364,     97,   1067,     96,    829,
          596,    929,  50353,   1569,    102,   1068,    611,    791,
          907,    793,   5444,   5396,    100,   2863,    790,    789,
          101,    639,    411,     91,   3226,    905,   3821,   2791,
          627,    614, 523531,   5848,   3936,    792,      0,    157,
         3610,   3216,   3480,    800,   1761,   5743,   5851,   3251,
         3556,    103,    797,   1045, 523530,   5319,   3509,    810,
         1209,   3058,   5246,   3217,   4334,   3362,    248,   3513,
         1059,    630,   1231,    641,    563,   4794,    168,    811,
           81,   5397,    803,   3482,   2659,   3064,    188,    105,
         3363,   5491,   3364,   4363,   4094,   1808,   3720, 523543,
         3360,   4346,   4376,   3361,   4340,   4342,   3031,   4344,
         3242,   4096,   3060,    862,   1483,    917,    184,    245,
         1028,   3697,   3698,   3703,   5298,  65535,  51923,   4607,
      

In [15]:
fault_diag['spn'].nunique()

420

In [16]:
fault_diag[fault_diag['spn'] == 5246]['EquipmentID'].nunique() 
# There are 191 equipment with spn/fault code 5246

191

In [17]:
fault_diag[(fault_diag['spn'] == 1569) & (fault_diag['fmi'] == 31)]['EquipmentID'].nunique() 

491

In [18]:
# creating list of equipment with partial derate
partial = fault_diag[(fault_diag['spn'] == 1569) & (fault_diag['fmi'] == 31)]['EquipmentID'].unique() 

In [19]:
fault_diag[fault_diag['spn'] == 5246]['EquipmentID'].nunique()

191

In [20]:
# creating list of unique EquipmentID with full derate. 
equipment = fault_diag[fault_diag['spn'] == 5246]['EquipmentID'].unique()

In [21]:
# To filter equipmnet ID that undergone partial derate but not full derate
def compare_arrays(arr1, arr2):
    non_matching = []
    for element in arr1:
        if element not in arr2:
            non_matching.append(element)
    for element in arr2:
        if element not in arr1:
            non_matching.append(element)
    return non_matching

arr1 = equipment
arr2 = partial
non_matching = compare_arrays(arr1, arr2)

In [22]:
non_matching

['1547',
 '1742',
 '305',
 '1535',
 '1524',
 '304',
 '1534',
 '1539',
 '301',
 '1549',
 '1529',
 '1745',
 '1525',
 '1537',
 '1521',
 '1528',
 '1543',
 '1749',
 '1746',
 '2143',
 '2111',
 '1748',
 '2209',
 '2082',
 '2109',
 '2089',
 '2211',
 '302',
 '2175',
 '1721',
 '1515',
 '1499',
 '1365',
 '1340',
 '1328',
 '1430',
 '1370',
 '1497',
 '1350',
 '1608',
 '1606',
 '1612',
 '1354',
 '1493',
 '1583',
 '1465',
 '1789',
 '1438',
 '1463',
 '1514',
 '1666',
 '1357',
 '310',
 '1660',
 '1445',
 '1494',
 '1367',
 '1364',
 '1740',
 '1425',
 '1358',
 '1646',
 '1428',
 '1346',
 '1562',
 '1433',
 '1424',
 '1771',
 '1790',
 '1596',
 '1421',
 '1480',
 '1466',
 '1457',
 '1703',
 '1626',
 '1715',
 '1597',
 '1610',
 '1377',
 '1652',
 '1517',
 '1784',
 '1786',
 '1341',
 '1382',
 '1714',
 '1369',
 '1398',
 '1791',
 '1772',
 '1645',
 '1475',
 '1806',
 '1607',
 '1450',
 '1368',
 '1650',
 '1495',
 '1587',
 '1670',
 '1617',
 '1476',
 '1441',
 '1405',
 '1577',
 '1447',
 '1408',
 '1439',
 '1446',
 '1629',
 '1730

In [23]:
equipment = np.sort(equipment,kind='quicksort')
equipment

array(['1329', '1339', '1366', '1373', '1375', '1378', '1383', '1384',
       '1389', '1391', '1395', '1396', '1399', '1401', '1403', '1407',
       '1417', '1418', '1419', '1431', '1437', '1440', '1443', '1444',
       '1452', '1453', '1458', '1472', '1473', '1477', '1486', '1487',
       '1488', '1490', '1492', '1501', '1508', '1509', '1519', '1521',
       '1524', '1525', '1528', '1529', '1534', '1535', '1537', '1539',
       '1543', '1547', '1549', '1551', '1552', '1554', '1556', '1557',
       '1559', '1560', '1561', '1563', '1564', '1566', '1567', '1571',
       '1572', '1573', '1575', '1579', '1581', '1582', '1584', '1585',
       '1586', '1590', '1591', '1592', '1595', '1598', '1599', '1600',
       '1601', '1602', '1603', '1604', '1605', '1623', '1630', '1637',
       '1643', '1654', '1657', '1659', '1661', '1663', '1665', '1668',
       '1669', '1683', '1686', '1689', '1691', '1692', '1698', '1704',
       '1711', '1718', '1739', '1742', '1743', '1745', '1746', '1748',
      

In [24]:
# filter rows based on values present in an equipment array
# filtering only for equipment that undergone full derate
fault_diag_filtered = fault_diag[fault_diag['EquipmentID'].isin(equipment)]

In [25]:
fault_diag_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
9,10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69
13,14,990470,2015-02-21 11:43:40.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1582,105410980,40.161944,-80.161018,2015-02-21 11:43:36.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
15,16,990495,2015-02-21 11:45:06.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1601,105430420,38.272037,-78.937129,2015-02-21 11:45:02.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051183,1248429,123886552,2020-03-06 12:11:54.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,False,126,NaN,1853,105376821,26.079814,-80.123518,2020-03-06 12:11:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,NaN
1051185,1248431,123891846,2020-03-06 12:20:36.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,True,NaN,0,NaN,100,0.58
1051186,1248432,123891847,2020-03-06 12:20:36.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,True,NaN,0,NaN,100,0.58
1051192,1248438,123893761,2020-03-06 12:51:53.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,False,126,NaN,1853,105376821,26.078888,-80.122361,2020-03-06 12:51:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# filter for only records where active is True
fault_diag_filtered = fault_diag_filtered[fault_diag_filtered['active'] == True]
fault_diag_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69
16,17,990504,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1601,105430420,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,False,66.48672,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,NaN,NaN,36.38905,3276.75,NaN,1.16
23,29,990653,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,49,929,9,True,126,NaN,1605,105415080,37.521898,-85.881759,2015-02-21 11:54:29.000,0,14.21,False,66.48672,231667.7,168.8,13,26.1,204.4063,599.875,4378.65,NaN,32180.911030536,0.594389,32,True,96.8,1279,True,NaN,0,3276.75,0,0.29
28,35,990771,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00187677*082113134117*07700053*I0*BBZ*,79619763,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1585,105443412,38.904675,-85.823657,2015-02-21 12:01:53.000,56.8,14.355,False,66.48672,319674.2,186.8,52,34.8,220.2688,1386.5,5791.95,29.2,48822.165274224,12.0727,NaN,True,82.4,1023,NaN,NaN,65.20514,3276.75,NaN,19.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051154,1248395,123872855,2020-03-06 10:46:24.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,25.994953,-80.182175,2020-03-06 10:57:23.000,0,14.645,False,66.48672,443424,185,12,24.94,193.8313,600.625,10755.3,68.8,62061.94017636,0.7264754,32,True,136.4,1279,True,NaN,0,NaN,100,0.29
1051179,1248425,123884064,2020-03-06 12:00:41.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,False,66.48672,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,True,NaN,0,NaN,100,0.58
1051180,1248426,123884065,2020-03-06 12:00:41.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,False,66.48672,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,True,NaN,0,NaN,100,0.58
1051185,1248431,123891846,2020-03-06 12:20:36.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92

In [27]:
fault_diag_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175781 entries, 5 to 1051186
Data columns (total 44 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   RecordID                   175781 non-null  int64  
 1   ESS_Id                     175781 non-null  int64  
 2   EventTimeStamp             175781 non-null  object 
 3   eventDescription           169000 non-null  object 
 4   actionDescription          0 non-null       float64
 5   ecuSoftwareVersion         146145 non-null  object 
 6   ecuSerialNumber            142116 non-null  object 
 7   ecuModel                   170841 non-null  object 
 8   ecuMake                    170841 non-null  object 
 9   ecuSource                  175781 non-null  int64  
 10  spn                        175781 non-null  int64  
 11  fmi                        175781 non-null  int64  
 12  active                     175781 non-null  bool   
 13  activeTransitionCount      1

In [28]:
# NaN values sum in each column
fault_diag_filtered.isnull().sum()

RecordID                          0
ESS_Id                            0
EventTimeStamp                    0
eventDescription               6781
actionDescription            175781
ecuSoftwareVersion            29636
ecuSerialNumber               33665
ecuModel                       4940
ecuMake                        4940
ecuSource                         0
spn                               0
fmi                               0
active                            0
activeTransitionCount             0
faultValue                   175781
EquipmentID                       0
MCTNumber                         0
Latitude                          0
Longitude                         0
LocationTimeStamp                 0
AcceleratorPedal              33195
BarometricPressure             5877
CruiseControlActive            8217
CruiseControlSetSpeed          7936
DistanceLtd                    5919
EngineCoolantTemperature       5946
EngineLoad                     5935
EngineOilPressure           

In [29]:
# drop columns actionDescription, faultValue and ServiceDistance as they contain only NaN values
fault_diag_filtered = fault_diag_filtered.drop(columns=['actionDescription', 'faultValue', 'ServiceDistance'])

In [41]:
fault_diag_filtered.head(20)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,13.6022,3276.75,NaN,6.67
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,64.82649,3276.75,NaN,17.69
16,17,990504,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,True,1,1601,105430420,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,False,66.48672,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,NaN,36.38905,3276.75,NaN,1.16
23,29,990653,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1605,105415080,37.521898,-85.881759,2015-02-21 11:54:29.000,0,14.21,False,66.48672,231667.7,168.8,13,26.1,204.4063,599.875,4378.65,NaN,32180.911030536,0.594389,32,True,96.8,1279,True,0,3276.75,0,0.29
28,35,990771,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,04993120*00187677*082113134117*07700053*I0*BBZ*,79619763,6X1u10D1500000000,CMMNS,0,111,17,True,1,1585,105443412,38.904675,-85.823657,2015-02-21 12:01:53.000,56.8,14.355,False,66.48672,319674.2,186.8,52,34.8,220.2688,1386.5,5791.95,29.2,48822.165274224,12.0727,NaN,True,82.4,1023,NaN,65.20514,3276.75,NaN,19.43
37,49,990990,2015-02-21 12:14:51.000,Incorrect Data Brake Signal Sensor 2,unknown,unknown,unknown,unknown,11,1068,2,True,127,1437,105355660,35.874166,-84.235462,2015-02-21 12:15:27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
42,56,991057,2015-02-21 12:18:38.000,Special Instructions System Diagnostic Code #1,unknown,unknown,unknown,unknown,11,611,14,True,127,305,105362919,33.673750,-84.340324,2015-02-21 12:19:14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
43,57,991058,2015-02-21 12:18:38.000,Special Instructions Wheel Sensor ABS Axle 2 Left,unknown,unknown,unknown,unknown,11,791,14,True,127,305,105362919,33.673750,-84.340324,2015-02-21 12:19:14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
44,58,991072,2015-02-21 12:03:15.000,Low (Severity Low) Engine Coolant Level,04993120*00187222*082113134117*07700053*I0*BBZ*,79619768,6X1u10D1500000000,CMMNS,0,111,17,True,1,1586,105410983,36.055416,-85.176111,2015-02-21 12:03:50.000,NaN,13.63,False,66.48672,283927.7,138.2,10,36.54,134.8813,660.875,5509.05,83.2,42060.15325918,1.149152,NaN,True,78.8,1023,NaN,0,3276.75,NaN,1.16
45,59,991076,2015-02-21 12:19:44.000,Low (Severity Low) Engine Coolant Level,04993120*00190983*060712145341*07700027*I0*BBZ*,79623054,6X1u10D1500000000,CMMNS,0,111,17,True,1,1600,105410840,36.111990,-84.024398,2015-02-21 12:19:06.000,NaN,14.1375,False,64.6226,248727.3,188.6,100,35.96,219.65,1471.625,4977.05,48.8,36158.681703526,17.30332,NaN,

In [31]:
fault_diag_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175781 entries, 5 to 1051186
Data columns (total 41 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   RecordID                   175781 non-null  int64  
 1   ESS_Id                     175781 non-null  int64  
 2   EventTimeStamp             175781 non-null  object 
 3   eventDescription           169000 non-null  object 
 4   ecuSoftwareVersion         146145 non-null  object 
 5   ecuSerialNumber            142116 non-null  object 
 6   ecuModel                   170841 non-null  object 
 7   ecuMake                    170841 non-null  object 
 8   ecuSource                  175781 non-null  int64  
 9   spn                        175781 non-null  int64  
 10  fmi                        175781 non-null  int64  
 11  active                     175781 non-null  bool   
 12  activeTransitionCount      175781 non-null  int64  
 13  EquipmentID                1

In [32]:
fault_diag_filtered.columns

Index(['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription',
       'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
       'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
       'LocationTimeStamp', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake', 'Speed',
       'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure'],
      dtype='object')

In [34]:
# changing data types
fault_diag_filtered['EquipmentID'] = fault_diag_filtered['EquipmentID'].astype(int)
#fault_diag_filtered['AcceleratorPedal'] = fault_diag_filtered['AcceleratorPedal'].astype(int)
#fault_diag_filtered['BarometricPressure'] = fault_diag_filtered['BarometricPressure'].astype(float)

In [40]:
fault_diag_filtered['BarometricPressure'] = fault_diag_filtered['BarometricPressure'].astype(float)

ValueError: could not convert string to float: '14,355'

In [37]:
fault_diag_filtered[fault_diag_filtered['AcceleratorPedal'] == '4,8']

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
77033,88003,2964433,2015-06-15 15:01:09.000,Data May Be Invalid Relative Speed; Rear Axle ...,PC4__1284P4C_2*,________Y048665,MX,PCAR,0,907,19,True,82,305,105362919,34.588935,-84.667962,2015-06-15 15:32:39.000,"4,8","14,355",False,0,"136467,8",185,0,"80,62","189,6687",1446,"2956,9",92,"21407,314319846",0,"116,6",True,131,17407,False,"28,92774","3276,75","6,8","17,98"


In [38]:
fault_diag_filtered['AcceleratorPedal'] = fault_diag_filtered['AcceleratorPedal'].replace('4,8', '4.8')


In [39]:
fault_diag_filtered['AcceleratorPedal'] = fault_diag_filtered['AcceleratorPedal'].astype(float)

ValueError: could not convert string to float: '51,2'

In [ ]:
fault_diag_filtered[fault_diag_filtered['spn'] == 5246]['EquipmentID'].nunique()

In [ ]:
fault_diag_filtered['spn'].unique()

In [42]:
fault_diag_filtered['ecuMake'].unique()

array(['CMMNS', 'unknown', 'PCAR', 'VOLVO', nan, '?CAR', '?MMNS',
       '?????MX', '??MNS', 'BNDWS', 'PACCR', '?????', '????R', '????S',
       'EATON', '?????MX16U13D13'], dtype=object)

In [ ]:
fault_diag_filtered['spn'].nunique()
# 311 different fault codes/ spn associated with equiments undergone full derate

In [ ]:
# fault_diag.loc[fault_diag['spn'].isin([5246, 1569])]

In [ ]:
# Counts of different faults 
fault_diag_filtered['spn'].value_counts()

In [ ]:
fault_diag_filtered['spn'].value_counts().loc[lambda x : x > 50] 

In [ ]:
fault_diag_filtered.groupby(['EquipmentID', 'spn'])['spn'].count()

In [ ]:
fault_diag_filtered.groupby('EquipmentID')['spn'].unique()

In [ ]:
fault_diag_filtered.groupby(['EquipmentID'])['spn'].count()


In [ ]:
fault_diag_filtered.groupby(['EquipmentID'])['spn'].count()

In [ ]:
fault_diag_filtered[fault_diag_filtered['spn'] == 5246]['EquipmentID'].value_counts().loc[lambda x : x > 1] 

In [ ]:
fault_diag_filtered.info()

In [ ]:
fault_diag_filtered.head(3)

In [ ]:
fault_diag_filtered.groupby(['EquipmentID'])['ecuSource'].unique()

In [ ]:
fault_diag_filtered[(fault_diag_filtered['EquipmentID'] == '1524') & (fault_diag_filtered['spn'] == 5246)]

In [52]:
# split location time stamp column
# convert the column to datetime format
fault_diag_filtered['EventTimeStamp'] = pd.to_datetime(fault_diag_filtered['EventTimeStamp'],format='%Y-%m-%d %H:%M:%S' )

# extract the date and time components into separate columns
fault_diag_filtered['date'] = fault_diag_filtered['EventTimeStamp'].dt.date
fault_diag_filtered['time'] = fault_diag_filtered['EventTimeStamp'].dt.time

# print the updated DataFrame
fault_diag_filtered


,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,date,time
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,13.6022,3276.75,NaN,6.67,2015-02-21,11:40:22
10,11,990459,2015-02-21 11:42:19,Low (Severity Low) Engine Coolant Level,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,64.82649,3276.75,NaN,17.69,2015-02-21,11:42:19
16,17,990504,2015-02-21 11:45:14,Low (Severity Low) Engine Coolant Level,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,True,1,1601,105430420,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,False,66.48672,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,NaN,36.38905,3276.75,NaN,1.16,2015-02-21,11:45:14
23,29,990653,2015-02-21 11:53:52,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1605,105415080,37.521898,-85.881759,2015-02-21 11:54:29.000,0,14.21,False,66.48672,231667.7,168.8,13,26.1,204.4063,599.875,4378.65,NaN,32180.911030536,0.594389,32,True,96.8,1279,True,0,3276.75,0,0.29,2015-02-21,11:53:52
28,35,990771,2015-02-21 12:01:10,Low (Severity Low) Engine Coolant Level,04993120*00187677*082113134117*07700053*I0*BBZ*,79619763,6X1u10D1500000000,CMMNS,0,111,17,True,1,1585,105443412,38.904675,-85.823657,2015-02-21 12:01:53.000,56.8,14.355,False,66.48672,319674.2,186.8,52,34.8,220.2688,1386.5,5791.95,29.2,48822.165274224,12.0727,NaN,True,82.4,1023,NaN,65.20514,3276.75,NaN,19.43,2015-02-21,12:01:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051154,1248395,123872855,2020-03-06 10:46:24,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,1853,105376821,25.994953,-80.182175,2020-03-06 10:57:23.000,0,14.645,False,66.48672,443424,185,12,24.94,193.8313,600.625,10755.3,68.8,62061.94017636,0.7264754,32,True,136.4,1279,True,0,NaN,100,0.29,2020-03-06,10:46:24
1051179,1248425,123884064,2020-03-06 12:00:41,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,1853,105376821,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,False,66.48672,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,True,0,NaN,100,0.58,2020-03-06,12:00:41
1051180,1248426,123884065,2020-03-06 12:00:41,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,1853,105376821,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,False,66.48672,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,True,0,NaN,100,0.58,2020-03-06,12:00:41
1051185,1248431,123891846,2020-03-06 12:20:36,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32

In [53]:
fault_diag_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175781 entries, 5 to 1051186
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   RecordID                   175781 non-null  int64         
 1   ESS_Id                     175781 non-null  int64         
 2   EventTimeStamp             175781 non-null  datetime64[ns]
 3   eventDescription           169000 non-null  object        
 4   ecuSoftwareVersion         146145 non-null  object        
 5   ecuSerialNumber            142116 non-null  object        
 6   ecuModel                   170841 non-null  object        
 7   ecuMake                    170841 non-null  object        
 8   ecuSource                  175781 non-null  int64         
 9   spn                        175781 non-null  int64         
 10  fmi                        175781 non-null  int64         
 11  active                     175781 non-null  bool   

In [63]:
fault_diag_filtered.sort_values('EventTimeStamp').groupby(['EquipmentID']).rolling('24H', on = 'EventTimeStamp').count()

AcceleratorPedal  BarometricPressure      EventTimeStamp  \
EquipmentID                                                                     
301         43524                 1.0                 1.0 2015-05-11 13:11:20   
            45265                 1.0                 1.0 2015-05-13 08:22:32   
            49800                 1.0                 1.0 2015-05-18 09:34:05   
            53829                 1.0                 1.0 2015-05-21 13:57:35   
            60312                 1.0                 1.0 2015-05-28 13:31:41   
...                               ...                 ...                 ...   
2211        1032513               1.0                 1.0 2019-12-13 05:10:57   
            1032642               2.0                 2.0 2019-12-13 10:55:35   
            1043663               1.0                 1.0 2020-02-03 10:37:54   
            1043727               2.0                 2.0 2020-02-03 15:46:46   
            1047567               1.0                 1.0 2020-02-19 05:43:54   

                     CruiseControlActive  CruiseControlSetSpeed  DistanceLtd  \
EquipmentID                                                                    
301         43524                    1.0                    1.0          1.0   
            45265                    1.0                    1.0          1.0   
            49800                    1.0                    1.0          1.0   
            53829                    1.0                    1.0          1.0   
            60312                    1.0                    1.0          1.0   
...                                  ...                    ...          ...   
2211        1032513                  1.0                    1.0          1.0   
            1032642                  2.0                    2.0          2.0   
            1043663                  1.0                    1.0          1.0   
            1043727                  2.0                    2.0          2.0   
            1047567                  1.0                    1.0          1.0   

                     ESS_Id  EngineCoolantTemperature  EngineLoad  \
EquipmentID                                                         
301         43524       1.0                       1.0         1.0   
            45265       1.0                       1.0         1.0   
            49800       1.0                       1.0         1.0   
            53829       1.0                       1.0         1.0   
            60312       1.0                       1.0         1.0   
...                     ...                       ...         ...   
2211        1032513     1.0                       1.0         1.0   
            1032642     2.0                       2.0         2.0   
            1043663     1.0                       1.0         1.0   
            1043727     2.0                       2.0         2.0   
            1047567     1.0                       1.0         1.0   

                     EngineOilPressure  EngineOilTemperature  EngineRpm  \
EquipmentID                                                               
301         43524                  1.0                   1.0        1.0   
            45265                  1.0                   1.0        1.0   
            49800                  1.0                   1.0        1.0   
            53829                  1.0                   1.0        1.0   
            60312                  1.0                   1.0        1.0   
...                                ...                   ...        ...   
2211        1032513                1.0                   1.0        1.0   
            1032642                2.0                   2.0        2.0   
            1043663                1.0                   1.0        1.0   
            1043727                2.0                   2.0        2.0   
            1047567                1.0                   1.0        1.0   

                     EngineTimeLtd  FuelLevel  FuelL

In [ ]:
fault_diag_filtered[(fault_diag_filtered['EquipmentID'] == '1524') & (fault_diag_filtered['spn'] == 5246)].groupby('date').count()

In [ ]:

# get the current time
current_time = datetime.datetime.now().time()

# set the time component of the datetime column to the current time
df['date_time'] = df['date_time'].apply(lambda x: datetime.datetime.combine(x.date(), current_time))

# print the updated DataFrame
print(df)


In [ ]:
with open ()